# System Design

In this notebook we note the operations performed by PVSyst in order to design the system.

It is necessary to indicate the kWp of the system at DC. The number of modules necessary is automatically calculated by PVSyst, given the characteristics of the PV modules selected.

The number of parallel modules is again calculated automatically.

The number of inverters needed is calculated by PVSyst, once again, automatically. The number of inverters is first calculated as

*n_inverters = dc_kwp / ac_inverter_nominal_power*

rounded up.

100kW
30kW

More details are taken into consideration, then. For each inverter, it calculates the maximum number of modules that can be connected in parallel to the inverter (number of strings). To do so, it takes into consideration the electrical limits of the inverter(s):

1. maximum input voltage
2. maximum input current
3. maximum and minimum voltage

An economic analysis is performed, minimizing LCOE, to understand whether it is better to undersize inverters, oversize them or keeping the initial number of inverters estimate.

Another useful KPI is the oversizing ratio:

*oversizing_ratio = dc_kwp / sum(ac_inverter_nominal_power)*

This parameter is often between 0.9 and 1.3. Having a ratio above 1 means enabling clipping. Such choice, though, is often accepted due to inverter's high costs.